In [116]:
import numpy as np
import pandas as pd

In [117]:
df = pd.read_csv('sales_pos.csv')

In [118]:
df1 = df.copy()
df1.shape

(550068, 11)

In [119]:
df1.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


In [120]:
# prod 별 각 도시의 purchase 평균, 표준편차

pv_df1 = pd.pivot_table(df1, index='prod',columns=['city'],values='purchase',aggfunc=['mean','std'])
pv_df1.head(2)

mean                                      std               \
city                  A             B             C            A            B   
prod                                                                            
P00000142  11077.560784  11174.903448  11150.681818  2283.546514  2280.121291   
P00000242   9945.843373  10377.618785  11282.517857  3681.578007  3028.912919   

                        
city                 C  
prod                    
P00000142  2344.426206  
P00000242  2865.131524

In [121]:
# B 도시에서 구매 평균이 가능 높은 상품은?
pv_df1['mean']['B'].idxmax()

'P00086242'

In [122]:
# df['상위','하위'] 로 selecting
# pv_df1.xs(('mean','B'),level=[0,1],axis=1)

pv_df1['mean','B'].idxmax()

'P00086242'

In [123]:
# stack : column 을 index 로 이동 
# pivot -> stack -> group 이랑 동일하게 만들수 있네.

pv_df1.stack(level=1)     # level=0 은 mean,std 가 인덱스로 이동

mean          std
prod      city                           
P00000142 A     11077.560784  2283.546514
          B     11174.903448  2280.121291
          C     11150.681818  2344.426206
P00000242 A      9945.843373  3681.578007
          B     10377.618785  3028.912919
...                      ...          ...
P0099842  B      6838.640000  1870.601690
          C      7606.575758  1336.883223
P0099942  A      6686.600000  1463.875609
          B      4712.875000  2072.996826
          C      6883.000000          NaN

[10321 rows x 2 columns]

In [124]:
# 이게 group 이랑 같은 거네.
pd.pivot_table(df1, index=['prod','city'],values='purchase',aggfunc=['mean','std'])

mean          std
                    purchase     purchase
prod      city                           
P00000142 A     11077.560784  2283.546514
          B     11174.903448  2280.121291
          C     11150.681818  2344.426206
P00000242 A      9945.843373  3681.578007
          B     10377.618785  3028.912919
...                      ...          ...
P0099842  B      6838.640000  1870.601690
          C      7606.575758  1336.883223
P0099942  A      6686.600000  1463.875609
          B      4712.875000  2072.996826
          C      6883.000000          NaN

[10321 rows x 2 columns]

In [125]:
# 똑 같이 groupby 로 만들어 보자

df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std'])

purchase             
                    sum          std
prod      city                      
P00000142 A     2824778  2283.546514
          B     4861083  2280.121291
          C     5151615  2344.426206
P00000242 A      825505  3681.578007
          B     1878349  3028.912919
...                 ...          ...
P0099842  B      341932  1870.601690
          C      251017  1336.883223
P0099942  A       33433  1463.875609
          B       37703  2072.996826
          C        6883          NaN

[10321 rows x 2 columns]

In [126]:
top_prod = df1.groupby(by=['prod'])['purchase'].sum().idxmax()
top_prod

'P00025442'

In [140]:
rule1 = df1['prod'].str[1:].astype('int')%4 != 0
df1.loc[rule1,:].groupby(by=['prod'])['purchase'].agg(['sum'])   # ('sum') 하면 Series 리턴

# prod 값이 4의 배수가 아닌 데이터중에서 (rule1) , prod별 purchase 의 합이  .idxmax() 가장 큰 prod ?

,sum
prod,
P00000142,12837476
P00000242,3967496
P00000342,1296475
P00000442,441173
P00000542,807212
...,...
P0099442,2870383
P0099642,83710
P0099742,991948


In [127]:
df1.loc[df1['prod'] == top_prod, 'job'].value_counts(normalize=True)

job
4     0.136842
7     0.115789
0     0.110836
17    0.088545
1     0.076780
12    0.072446
14    0.052012
2     0.048297
20    0.046440
16    0.037152
10    0.034056
6     0.033437
15    0.030960
3     0.028483
11    0.023529
5     0.019195
13    0.014861
19    0.012384
18    0.008669
9     0.007430
8     0.001858
Name: proportion, dtype: float64